## ReTap - UPDRS-Tapping Assessment - Feature Exploration

This notebooks helps to visualise and statistically tst created tapping-features.



### 0. Loading packages and functions, defining paths



In [1]:
# Importing Python and external packages
import os
import sys
import importlib
import pandas as pd
import numpy as np
import sklearn as sk
import scipy
import matplotlib.pyplot as plt



In [2]:
# check some package versions for documentation and reproducability
print('Python sys', sys.version)
print('pandas', pd.__version__)
print('numpy', np.__version__)
# print('mne_bids', mne_bids.__version__)
# print('mne', mne.__version__)
print('sci-py', scipy.__version__)
print('sci-kit learn', sk.__version__)



## developed with:
# Python sys 3.9.7 (default, Sep 16 2021, 08:50:36) 
# [Clang 10.0.0 ]
# pandas 1.3.4
# numpy 1.20.3
# mne_bids 0.9
# mne 0.24.1
# sci-py 1.7.1
# sci-kit learn 1.0.1

## Currently (own env) since 31.08.22
# Python sys 3.9.12 (main, Jun  1 2022, 06:36:29) 
# [Clang 12.0.0 ]
# pandas 1.4.3
# numpy 1.21.5
# sci-py 1.7.3
# sci-kit learn 1.1.1

Python sys 3.9.13 (main, Oct 13 2022, 21:23:06) [MSC v.1916 64 bit (AMD64)]
pandas 1.4.4
numpy 1.23.3
sci-py 1.9.1
sci-kit learn 1.1.2


In [49]:
# ft extraction
import tap_extract_fts.tapping_postFeatExtr_calc as ft_calc 

# own helper functions
import retap_utils.utils_dataManagement as utils_dataMn

import tap_plotting.main_plot_single_tap_timings as plot_timings

### Load or creating tapping-traces

In [4]:
### IMPORT CREATED CLASSES FROM FILES
from tap_extract_fts.main_featExtractionClass import FeatureSet, singleTrace

deriv_path = os.path.join(utils_dataMn.get_local_proj_dir(), 'data', 'derivatives')

# berClass = utils_dataManagement.load_class_pickle(os.path.join(deriv_path, 'ftClass_bertest.P'))
# dusClass = utils_dataManagement.load_class_pickle(os.path.join(deriv_path, 'ftClass_DUS.P'))

fts = utils_dataMn.load_class_pickle(os.path.join(deriv_path, 'ftClass_ALL_20230301.P'))
# ftClass = utils_dataMn.load_class_pickle(os.path.join(deriv_path, 'ftClass_ALL_20221214.P'))
# ftClass10 = utils_dataMn.load_class_pickle(os.path.join(deriv_path, 'ftClass_ALL_max10_20221214.P'))

### 1 Visualise detected Features over time (FIG3)

In [80]:
importlib.reload(ft_calc)

fig_fname = 'FIG3_featCourse_2Cases'

PLOT_SINGLE_TRACES = True
PLOT_MEANS = True
TO_SAVE = False

subs = [
    'BER023',
    'BER024',
    # 'DUS022',
    # 'DUS007'
]
ft_sel = ['impactRMS', 'raise_velocity', 'intraTapInt', 'tap_entropy']
ft_names = ['impact-RMS', 'raising-velocity','inter-tap-interval',  'tap-entropy']
ft_units = ['g', 'm/s/s', 's',  'a.u.']

score_colors = {0: 'olivedrab', 1: 'forestgreen',  #lawngreen
                2: 'blue', 3: 'purple'}
fsize = 16 
fig, axes = plt.subplots(len(ft_sel), len(subs), 
                         sharex='col',
                        #  sharey='row',
                         figsize=(12, 12))

for i_s, sub in enumerate(subs):
    if i_s == 0: title = 'Case A'
    elif i_s == 1: title = 'Case B'
    axes[0, i_s].set_title(title, fontsize=fsize+4, weight='bold',)

    for i_f, f in enumerate(ft_sel):

        axes[i_f, 0].set_ylabel(r"$\bf{" + str(ft_names[i_f])
                                + "}$" + f'\n[{ft_units[i_f]}]',
                                fontsize=fsize,)

        # gather score values per sub, per feat
        if PLOT_MEANS:
            score_lists = {}
            for score in score_colors.keys(): score_lists[score] = []

        for t in fts.incl_traces:

            if not t.startswith(sub): continue
            score = getattr(fts, t).tap_score
            if score == 4: continue
            col = score_colors[score]

            feats = getattr(fts, t).fts
            values = getattr(feats, f)

            if PLOT_SINGLE_TRACES:
                axes[i_f, i_s].plot(values, color=col, alpha=.2,)
            
            if PLOT_MEANS:
                 score_lists[score].append(list(values))

        if PLOT_MEANS:
            mean_dict, err_dict = ft_calc.get_means_std_errs(score_lists)
            for score in mean_dict.keys():
                axes[i_f, i_s].plot(mean_dict[score],
                                    color=score_colors[score],
                                    alpha=.8, lw=3,)
                axes[i_f, i_s].fill_between(x=np.arange(len(mean_dict[score])),
                                            y1=mean_dict[score] - err_dict[score],
                                            y2=mean_dict[score] + err_dict[score],
                                            color=score_colors[score], alpha=.3,)
                


        axes[i_f, i_s].tick_params(axis='both', labelsize=fsize,
                                    size=fsize)
        axes[i_f, i_s].spines[['right', 'top']].set_visible(False)

    axes[-1, i_s].set_xlabel('Detected taps (observations)', fontsize=fsize,)
    # create legend labels with dummy plots
    if i_s == 0:
        for score in score_colors.keys():
            axes[-1, 1].plot([], c=score_colors[score], label=f"{score}'s",
                             lw=5, )

        lgd = fig.legend(ncol=4, fontsize=fsize+4, frameon=False,
                   bbox_to_anchor=(.5, -.05), loc='lower center')


plt.tight_layout()

if TO_SAVE:
    plt.savefig(os.path.join(utils_dataMn.find_onedrive_path('figures'),
                             'feature_course (fig3)', fig_fname),
                bbox_extra_artists=(lgd,), bbox_inches='tight',
                dpi=450, facecolor='w',)
plt.close()

In [ ]:
import tap_plotting.retap_check_taps as plot_taps

In [ ]:
# ### PLOT DETECTED TAPS for all Traces and save figures
# importlib.reload(plot_taps)
# plot_taps.plot_detected_taps(ftClass)

### 2 Individual feature differences between therapeutic conditions

In [6]:
SUBS_EXCL = ['BER028', ]  # too many missing acc-data
TRACES_EXCL = [
    'DUS006_M0S0_L_1',  # no score/video
    'DUS017_M1S0_L_1', 'DUS017_M1S1_L_1',  # corrupt acc-axis
    # 'BER023_M1S0_R_2',  # tap score 4
]
for sub in SUBS_EXCL:
    for t in fts.incl_traces:
        if t.startswith(sub):
            print(t)
            TRACES_EXCL.append(t)

In [16]:
TRACES = []  # traces to include

for t in fts.incl_traces:
    if t not in TRACES_EXCL:
        TRACES.append(t)

SUBS = [t[:6] for t in TRACES]
SUBS = list(set(SUBS))


In [210]:
def select_best_worst_traces(
    traces_consider: list, method='per_hand',
    only_responders=False,  
):
    subs_consider = [t[:6] for t in traces_consider]
    subs_consider = list(set(subs_consider))

    TRACE_SEL = {}

    if method == 'per_state':
        TRACE_SEL['best'] = []
        TRACE_SEL['worst'] = []

    for SUB in subs_consider:
        # handle sides separately
        for SIDE in ['L', 'R']:
            # DUS never right, only left recorded
            if SUB.startswith('DUS') and SIDE == 'R': continue

            if method == 'per_hand':
                TRACE_SEL[f'{SUB}_{SIDE}'] = {'best': [], 'worst': []}

            # select traces
            for t in traces_consider:
                sub, cnd, side, run = t.split('_')

                if sub != SUB or side != SIDE: continue

                # select condition
                if cnd == 'M0S0':
                    if method == 'per_hand':
                        TRACE_SEL[f'{SUB}_{SIDE}']['worst'].append(t)
                    elif method == 'per_state':
                        TRACE_SEL['worst'].append(t)
                if cnd == 'M1S1':
                    if method == 'per_hand':
                        TRACE_SEL[f'{SUB}_{SIDE}']['best'].append(t)
                    elif method == 'per_state':
                        TRACE_SEL['best'].append(t)

            # select out sub-sides without best and worst present
            if method == 'per_hand':
                if (TRACE_SEL[f'{SUB}_{SIDE}']['best'] == [] or
                    TRACE_SEL[f'{SUB}_{SIDE}']['worst'] == []):

                    del TRACE_SEL[f'{SUB}_{SIDE}']
            
            # elif method == 'per_state':
            #     if only_responders:
                    

    return TRACE_SEL

In [206]:
def get_subsides_responders(
    traces_consider: list, min_diff=1,
    
):
    n_excl = 0
    subsides_consider = []

    for t in traces_consider:
        
        sub, cnd, side, run = t.split('_')
        if [sub, side] not in subsides_consider:
            subsides_consider.append([sub, side])

    responders = {}

    for SUB, SIDE in subsides_consider:

        state_scores = {'M1S1': [], 'M1S0': [], 'M0S1': [], 'M0S0': []}

        for t in traces_consider:
            sub, cnd, side, run = t.split('_')
            if not (sub == SUB and SIDE == side): continue
            state_scores[cnd].append(getattr(fts, t).tap_score)
        
        # get means per state
        off_mean = np.nanmean(state_scores['M0S0'])
        if np.isnan(off_mean):
            print(f'NO OFF: skip {SUB, SIDE}')
            n_excl += 1
            continue
        on_means = []
        for state in ['M1S1', 'M1S0', 'M0S1']:
            on_means.append(np.nanmean(state_scores[state]))
        if np.isnan(on_means).all():
            n_excl += 1
            print(f'NO ONs: skip {SUB, SIDE}')
            continue
            

        if (off_mean - np.nanmin(on_means)) >= min_diff:
            best_state = ['M1S1', 'M1S0', 'M0S1'][np.nanargmin(on_means)]
            responders[f'{SUB}_{SIDE}'] = best_state
        # else:
        #     print(off_mean, on_means)
    
    print(f'n responders: {len(responders)}, out of '
          f'{len(subsides_consider) - n_excl} ({n_excl} excl)')

    return responders


In [207]:
resp = get_subsides_responders(TRACES, min_diff=0.5)
print(resp)


NO OFF: skip ('BER021', 'L')
NO OFF: skip ('BER021', 'R')
NO ONs: skip ('BER056', 'R')
NO OFF: skip ('DUS006', 'L')
NO OFF: skip ('DUS009', 'L')
NO OFF: skip ('DUS023', 'L')
n responders: 25, out of 47 (6 excl)
{'BER025_R': 'M0S1', 'BER026_L': 'M1S0', 'BER029_L': 'M1S1', 'BER029_R': 'M1S1', 'BER033_L': 'M0S1', 'BER033_R': 'M0S1', 'BER052_L': 'M1S0', 'BER055_L': 'M1S0', 'BER055_R': 'M1S0', 'BER049_R': 'M1S0', 'BER023_L': 'M1S1', 'BER023_R': 'M0S1', 'DUS024_L': 'M1S1', 'DUS007_L': 'M1S1', 'DUS011_L': 'M1S1', 'DUS014_L': 'M1S1', 'DUS010_L': 'M1S1', 'DUS026_L': 'M1S1', 'DUS020_L': 'M1S1', 'DUS008_L': 'M1S1', 'DUS016_L': 'M1S1', 'DUS025_L': 'M1S1', 'DUS015_L': 'M1S1', 'DUS004_L': 'M1S1', 'DUS022_L': 'M1S1'}


C:\Users\habetsj\AppData\Local\Temp\ipykernel_5128\1791937933.py:33: RuntimeWarning: Mean of empty slice
  on_means.append(np.nanmean(state_scores[state]))
C:\Users\habetsj\AppData\Local\Temp\ipykernel_5128\1791937933.py:26: RuntimeWarning: Mean of empty slice
  off_mean = np.nanmean(state_scores['M0S0'])


In [258]:
five_fts = ['trace_RMSn',
            'jerkiness_trace',
            'coefVar_intraTapInt',
            'coefVar_impactRMS',
            'mean_raise_velocity']
ft_labels = ['normed RMS (trace)',
             'jerkiness (trace)',
             'CV ITI',
             'CV impact-RMS',
             'mean\nfinger-open velocity']

In [109]:
# trace_sel = select_best_worst_traces(TRACES, method='per_hand')


In [221]:
from scipy.stats import mannwhitneyu

In [287]:
# fig, axes = plt.subplots(len(five_fts), 1, figsize=(6, 12))
fig, ax = plt.subplots(1, 1, figsize=(16, 4))

cmaps = plot_timings.get_colors()

col_off = list(cmaps.values())[2]
col_all_on = list(cmaps.values())[5]
col_sel_on = list(cmaps.values())[1]
fs=18
signs_sel, signs_all = [], []

box_offs, box_on_sel, box_on_all = [], [], []

for i_ft, ft in enumerate(five_fts):
    # create benchmark variation on all
    sel_on_values, off_values, all_on_values = [], [], []

    
    for sub_side in resp.keys():
        best_cond = resp[sub_side]
        SUB, SIDE = sub_side.split('_')
    
        for t in TRACES:
            sub, cnd, side, run = t.split('_')
            if not (sub==SUB and side==SIDE): continue
            
            # fill off off
            if cnd == 'M0S0':
                trace_feats = getattr(fts, t).fts
                trace_value = getattr(trace_feats, ft)
                off_values.append(trace_value)
            # fill ALL other ONs
            else:
                all_on_values.append(trace_value)
            # fill selected responders
            if cnd == best_cond:
                trace_feats = getattr(fts, t).fts
                trace_value = getattr(trace_feats, ft)
                sel_on_values.append(trace_value)
                        
    
    # plot on one lines (normalised)
    box_offs.append(np.array(off_values) / max(off_values))
    box_on_all.append(np.array(all_on_values) / max(all_on_values))
    box_on_sel.append(np.array(sel_on_values) / max(sel_on_values))

    p_sel = mannwhitneyu(off_values, sel_on_values)[1]
    p_all = mannwhitneyu(off_values, all_on_values)[1]
    if p_sel < (0.001 / len(five_fts)):
        signs_sel.append(True)
    else:
        signs_sel.append(False)
    if p_all < (0.001 / len(five_fts)):
        signs_all.append(True)
    else:
        signs_all.append(False)
    

    
bp1 = ax.boxplot(box_offs, positions=np.arange(0, 5, 1),
                 widths=0.15, patch_artist=True)
bp2 = ax.boxplot(box_on_all, positions=np.arange(0.2, 5.2, 1),
                 widths=0.15, patch_artist=True)
bp3 = ax.boxplot(box_on_sel, positions=np.arange(0.4, 5.4, 1),
                 widths=0.15, patch_artist=True,)
for b in bp1['boxes']: b.set(facecolor=col_off)
for b in bp2['boxes']: b.set(facecolor=col_all_on)
for b in bp3['boxes']: b.set(facecolor=col_sel_on)
plt.setp(bp1['medians'], color='k')
plt.setp(bp2['medians'], color='k')
plt.setp(bp3['medians'], color='k')

# add significance
for i_sig, sig in enumerate(signs_sel):
    if sig: plt.text(x=.4 + i_sig, y=1.1, s='*',
                     size=fs*2, ha='center', va='center',
                     color='red',)

ax.set_xticks(np.arange(.2, 5.2, 1))
ax.set_xticklabels(ft_labels)

ax.set_ylabel('Normalised feature\nvalue (a.u.)', size=fs)
ax.set_ylim(0, 1.7)
ax.set_yticks([0, 1])
ax.spines[['right', 'top',]].set_visible(False)  # 

ax.legend([bp1["boxes"][0], bp2["boxes"][0], bp3["boxes"][0]],
          [f"all Off-Off (n={len(box_offs[0])})",
           f"all On (n={len(box_on_all[0])})",
           f"best On (n={len(box_on_sel[0])})"],
           ncol=3, fontsize=fs + 4,
          loc='upper center', frameon=False,)

plt.tick_params(axis='both', size=fs, labelsize=fs,
                color='k')
plt.tight_layout()

plt.savefig(os.path.join(utils_dataMn.find_onedrive_path('figures'),
                         'SuppFig_OnOff_differences'),
            dpi=450, facecolor='w',)

plt.close()



In [55]:
cmaps

{'nightblue': '#332288',
 'darkgreen': '#117733',
 'turquoise': '#44AA99',
 'lightblue': '#88CCEE',
 'sand': '#DDCC77',
 'softred': '#CC6677',
 'lila': '#AA4499',
 'purplered': '#882255'}